<a href="https://colab.research.google.com/github/ramya-2906/Medicine-Recommendation-System/blob/main/Medicine_Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# -------------------------------
# Step 1: Install Required Libraries
# -------------------------------
!pip install pandas scikit-learn

In [16]:
# -------------------------------
# Step 2: Import Libraries
# -------------------------------
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [17]:
# -------------------------------
# Step 3: Load Dataset
# -------------------------------
df = pd.read_csv('medicine.csv')  # replace with your CSV path

# Inspect dataset
print("Columns in dataset:", df.columns)
print("Dataset preview:")
print(df.head())

Columns in dataset: Index(['index', 'Drug_Name', 'Reason', 'Description'], dtype='object')
Dataset preview:
   index                                          Drug_Name Reason  \
0      1               A CN Gel(Topical) 20gmA CN Soap 75gm   Acne   
1      2  A Ret 0.05% Gel 20gmA Ret 0.1% Gel 20gmA Ret 0...   Acne   
2      3                             ACGEL CL NANO Gel 15gm   Acne   
3      4                                ACGEL NANO Gel 15gm   Acne   
4      5                              Acleen 1% Lotion 25ml   Acne   

                                         Description  
0                      Mild to moderate acne (spots)  
1  A RET 0.025% is a prescription medicine that i...  
2  It is used to treat acne vulgaris in people 12...  
3  It is used to treat acne vulgaris in people 12...  
4  treat the most severe form of acne (nodular ac...  


In [18]:
# -------------------------------
# Step 4: Preprocess Data
# -------------------------------
# Drop rows with missing values in essential columns
df = df.dropna(subset=['Drug_Name', 'Reason', 'Description'])

# Create a lowercase version of the Reason column for case-insensitive matching
df['Reason_lower'] = df['Reason'].str.lower()


In [19]:
# -------------------------------
# Step 5: Build TF-IDF Matrix
# -------------------------------
# We'll use the 'Description' column to find similarity between diseases/reasons
tfidf = TfidfVectorizer(stop_words='english')
tfidf_matrix = tfidf.fit_transform(df['Description'])

# Compute cosine similarity matrix
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)


In [20]:
# -------------------------------
# Step 6: Recommendation Function
# -------------------------------
def recommend_medicine(reason_name):
    reason_lower = reason_name.lower()

    if reason_lower not in df['Reason_lower'].values:
        return "Reason/Disease not found in dataset."

    # Get index of the reason
    idx = df[df['Reason_lower'] == reason_lower].index[0]

    # Get similarity scores
    sim_scores = list(enumerate(cosine_sim[idx]))

    # Sort by similarity
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)

    # Take top 5 similar reasons (excluding itself)
    sim_scores = sim_scores[1:6]

    # Get indices
    indices = [i[0] for i in sim_scores]

    # Return recommended Drug_Names
    return df['Drug_Name'].iloc[indices].values


In [ ]:
# -------------------------------
# Step 7: Interactive Testing
# -------------------------------
while True:
    user_input = input("Enter a reason/disease (or type 'exit' to quit): ")
    if user_input.lower() == 'exit':
        print("Goodbye 👋")
        break
    recommendations = recommend_medicine(user_input)
    print(f"\nRecommended drugs for '{user_input}':\n{recommendations}\n")

Enter a reason/disease (or type 'exit' to quit): acne

Recommended drugs for 'acne':
['Acnedap Gel 15gm' "Acnetoin 20mg Capsule 10'SAcnetoin Gel 15gm"
 'Acnin Pimple Care Face Pack 50gm' 'Adapnil Gel 15gm' 'Alene Gel 15gm']

Enter a reason/disease (or type 'exit' to quit): cold

Recommended drugs for 'cold':
Reason/Disease not found in dataset.

Enter a reason/disease (or type 'exit' to quit): fever

Recommended drugs for 'fever':
['Aalcetamol 1000mg Infusion 100ml' 'ACETO 250mg Syrup 60ml'
 'Agpar 125mg Suspension 450ml' "ALGINA 500MG TABLET 10'S"
 "AMIDOL Tablet 10's"]

Enter a reason/disease (or type 'exit' to quit): dandruff

Recommended drugs for 'dandruff':
['Dankare Plus Scalp Solution 100ml' 'Fongitar Liquid 100ml'
 'Keraglo AD Lotion 50mlKeraglo AD Shampoo 75ml' 'Ketafung CT Lotion 90ml'
 'Ketomac Shampoo 110ml']

